## UAV fleet Simulation 

This is for testing propouses

In [1]:
import os
import numpy as np

# Custom modules
from vehicle_logic import VehicleLogic
from helpers.change_coordinates import local2global
from helpers.visualization.plots import plot_3d_interactive
from plan import Plan,State
from gcs import GCS
from simulators.QGroundControl.qgc import QGC

#Kill all related process
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")

## Create Plans

In [2]:
offsets = [ #east, north, up, heading
            (0, 0, 0, 0)]

n_uavs = len(offsets)
local_paths=[Plan.create_square_path(side_len=dist,alt= 5) for dist in (5,7,10,12,15)]

homes=np.array([offset[:3] for offset in offsets])
gcs=GCS(positions=homes.copy())

global_paths=[local2global(np.array(path), home,pairwise=True) for path,home in zip(local_paths,homes)]

plans=[Plan.basic(wps=path,alt=5,wp_margin=0.5) for path in local_paths]


colors=['Green','Blue','Red','Orange','Yellow',]
markers = {f'uav_{i+1}': {'pos':global_paths[i],'color':colors[i]} for i in range(n_uavs)}

plot_3d_interactive(markers,title='Simulation Markers',expand=[0.2,0.2,0.6],ground=-0.05)



Plan 'basic' created — no connection yet 🧩
Action 'Pre-Arm Check' created — no connection yet 🧩
Step 'Check disarmed' created — no connection yet 🧩
Step 'Check EKF' created — no connection yet 🧩
Step 'Check GPS' created — no connection yet 🧩
Step 'Check system' created — no connection yet 🧩
Action 'Set Mode: GUIDED' created — no connection yet 🧩
Step 'Switch to GUIDED' created — no connection yet 🧩
Action 'Arm' created — no connection yet 🧩
Step 'arm' created — no connection yet 🧩
Action 'takeoff' created — no connection yet 🧩
Step 'takeoff' created — no connection yet 🧩
Action 'fly' created — no connection yet 🧩
Step 'go to  -> (0, 0, 5)' created — no connection yet 🧩
Step 'go to  -> (0, 5, 5)' created — no connection yet 🧩
Step 'go to  -> (5, 5, 5)' created — no connection yet 🧩
Step 'go to  -> (5, 0, 5)' created — no connection yet 🧩
Step 'go to  -> (0, 0, 5)' created — no connection yet 🧩
Action 'Land' created — no connection yet 🧩
Step 'land' created — no connection yet 🧩
Plan 'ba

## Choose Simulator

In [3]:
simulator=QGC(offsets=offsets,origin=(-35.3633245,149.1652241,0,0))

Launch Vehicles

In [4]:
simulator.launch_vehicles()

Create UAVs

In [5]:
uavs=[]
for i,(plan,home) in enumerate(zip(plans,homes)): 
    uavs.append(VehicleLogic(sys_id=i+1,
                    home=home,
                    plan= plan))

Vehicle 1: Step 'Check disarmed' is now connected ✅🔗
Vehicle 1: Step 'Check EKF' is now connected ✅🔗
Vehicle 1: Step 'Check GPS' is now connected ✅🔗
Vehicle 1: Step 'Check system' is now connected ✅🔗
Vehicle 1: Action 'Pre-Arm Check' is now connected ✅🔗
Vehicle 1: Action 'Pre-Arm Check' is now connected ✅🔗
Vehicle 1: Step 'Switch to GUIDED' is now connected ✅🔗
Vehicle 1: Action 'Set Mode: GUIDED' is now connected ✅🔗
Vehicle 1: Action 'Set Mode: GUIDED' is now connected ✅🔗
Vehicle 1: Step 'arm' is now connected ✅🔗
Vehicle 1: Action 'Arm' is now connected ✅🔗
Vehicle 1: Action 'Arm' is now connected ✅🔗
Vehicle 1: Step 'takeoff' is now connected ✅🔗
Vehicle 1: Action 'takeoff' is now connected ✅🔗
Vehicle 1: Action 'takeoff' is now connected ✅🔗
Vehicle 1: Step 'go to  -> (0, 0, 5)' is now connected ✅🔗
Vehicle 1: Step 'go to  -> (0, 5, 5)' is now connected ✅🔗
Vehicle 1: Step 'go to  -> (5, 5, 5)' is now connected ✅🔗
Vehicle 1: Step 'go to  -> (5, 0, 5)' is now connected ✅🔗
Vehicle 1: Step 'go

Launch Simulator

In [6]:
simulator.launch()

🧼 All QGroundControl links deleted (count=0).
✅ 1 new connection(s) added starting from port 5763. Restart QGroundControl to see them.


## Execute Plan

In [7]:
while gcs.incomplete_missions:
    to_remove = set()
    for id in gcs.incomplete_missions:
        uav = uavs[id-1]
        if uav.plan.state == State.DONE:
            to_remove.add(id)
        else:
            uav.plan.act()
    gcs.update_missions(to_remove)

Vehicle 1: ▶️ Starting Step: Check disarmed
Vehicle 1: ✅ Step: Check disarmed is done
Vehicle 1: ▶️ Starting Step: Check EKF
Vehicle 1: ✅ Step: Check EKF is done
Vehicle 1: ▶️ Starting Step: Check GPS
Vehicle 1: ✅ Step: Check GPS is done
Vehicle 1: ▶️ Starting Step: Check system
Vehicle 1: ✅ Step: Check system is done
Vehicle 1: ▶️ Starting Step: Switch to GUIDED
Vehicle 1: ✅ Step: Switch to GUIDED is done
Vehicle 1: ▶️ Starting Step: arm
Vehicle 1: ✅ Step: arm is done
Vehicle 1: ▶️ Starting Step: takeoff
Vehicle 1:📍 Distance to target: 5.02 m
Vehicle 1:📍 Distance to target: 5.02 m
Vehicle 1:📍 Distance to target: 5.02 m
Vehicle 1:📍 Distance to target: 5.02 m
Vehicle 1:📍 Distance to target: 5.02 m
Vehicle 1:📍 Distance to target: 5.02 m
Vehicle 1:📍 Distance to target: 5.02 m
Vehicle 1:📍 Distance to target: 5.02 m
Vehicle 1:📍 Distance to target: 5.02 m
Vehicle 1:📍 Distance to target: 5.02 m
Vehicle 1:📍 Distance to target: 5.02 m
Vehicle 1:📍 Distance to target: 5.02 m
Vehicle 1:📍 Distance 

## Kill all related process

In [8]:
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")